In [1]:
import os
os.environ['SPARK_HOME'] = '/opt/spark/spark-3.0.1-bin-hadoop2.7'

import findspark
findspark.init()

import pyspark
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.sql import SparkSession,SQLContext, Row, functions
from pyspark.sql.functions import asc, count, col, collect_list
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.clustering import LDA

import numpy as np
#spark.sparkContext.addPyFile("/home/giangvdq/workspaces/pythie/thesis/configuration.py")
import configuration


spark = SparkSession.builder.master("local[*]") \
                    .appName('Topic Modelling') \
                    .config("spark.driver.memory", "7g") \
                    .master("local[*]") \
                    .getOrCreate()

In [2]:
spark.sparkContext.uiWebUrl

'http://192.168.1.8:4040'

In [5]:
path = configuration.PATHS["merged"]["nytimes"]
df = spark.read.parquet(path)

In [7]:
df.show()

+------+--------------------+-----+
| docId|                word|count|
+------+--------------------+-----+
|298614|           primitive|    1|
|212296|           zzz_aaron|    1|
|  6683|            greatest|    1|
| 45054|                 big|    3|
|287251|            advisory|    1|
| 39800|              cringe|    1|
|188348|             leaving|    1|
|113678|      zzz_ohio_state|    6|
| 48732|          commission|    2|
|102503|           impolitic|    1|
| 75465|              debate|   11|
|235785|               shape|    1|
|  9120|             program|    1|
|292694|zzz_putnam_invest...|    1|
| 37634|           situation|    1|
|278220|               merit|    1|
|202393|                prod|    1|
| 91869|             debuted|    1|
| 65177|                yell|    1|
|163995|            anecdote|    1|
+------+--------------------+-----+
only showing top 20 rows



In [8]:
def parse(row):
    docId = row['docId']
    word = row['word']
    count = row['count']
    a = np.array([])
    for i in range(len(word)):
        lis = np.repeat(str(word[i]), count[i])
        a = np.concatenate((a,lis))
        normal_arr = a.tolist()
    row_info = Row(docId = docId, words =  normal_arr) #  Vectors.dense(a))
    return row_info

In [13]:
def pre_process(df):
    mergedDF = (df.groupBy("docId").
                    agg(collect_list("word").alias("word"), 
                        collect_list("count").alias("count")) ).cache()
    docId_rdd = mergedDF.rdd.map(lambda row:row[0]).cache()
    word_rdd = mergedDF.rdd.map(lambda row:row[1])
    count_rdd = mergedDF.rdd.map(lambda row:row[2])
    mergedDF2 = docId_rdd.zip(word_rdd.map(lambda x:Vectors.dense(x))).toDF(schema=['docId','word'])
    mergedDF3 = docId_rdd.zip(count_rdd.map(lambda x:Vectors.dense(x))).toDF(schema=['docId','count'])
    mergedDF_final = mergedDF2.alias('a').join(mergedDF3.alias('b'), ['docId']).select('a.docId','word','count')
    return mergedDF_final

In [14]:
def pre_process2(mergedDF_final):
    vectorRDD = mergedDF_final.rdd.map(lambda row: parse(row))
    vectorDF = vectorRDD.toDF()
    return vectorDF

In [15]:
mergedDF_final = pre_process(df)
mergedDF_final.printSchema()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 13, 192.168.1.8, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/spark-3.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/opt/spark/spark-3.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/spark/spark-3.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 223, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark/spark-3.0.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/spark/spark-3.0.1-

In [ ]:
mergedDF_final.show()

In [ ]:
vectorDF = pre_process2(mergedDF_final)
vectorDF.printSchema()

In [12]:
#vectorDF.coalesce(8).write.mode('overwrite').parquet("/home/giangvdq/workspaces/pythie/preprocess")

In [13]:
spark.read.parquet("/home/giangvdq/workspaces/pythie/preprocess").show()

+------+--------------------+
| docId|               words|
+------+--------------------+
|100016|[453.0, 674.0, 86...|
|100119|[66.0, 396.0, 396...|
|100231|[243.0, 442.0, 45...|
|100470|[442.0, 444.0, 45...|
|100513|[9.0, 254.0, 342....|
|100598|[265.0, 408.0, 63...|
|100762|[220.0, 379.0, 40...|
|100949|[86.0, 258.0, 376...|
|100978|[59.0, 263.0, 107...|
| 10154|[413.0, 1300.0, 1...|
|101683|[121.0, 157.0, 15...|
|102011|[317.0, 376.0, 60...|
|102033|[376.0, 376.0, 39...|
|102441|[59.0, 121.0, 376...|
|102590|[1135.0, 1224.0, ...|
|103082|[293.0, 400.0, 40...|
|103269|[444.0, 598.0, 21...|
|103295|[397.0, 458.0, 45...|
|103363|[458.0, 764.0, 76...|
|103958|[513.0, 743.0, 95...|
+------+--------------------+
only showing top 20 rows



In [15]:
from pyspark.ml.feature import CountVectorizer

Vector = CountVectorizer(inputCol="words", outputCol="vectors")
model = Vector.fit(vectorDF)
result = model.transform(vectorDF)

result.printSchema()

root
 |-- docId: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vectors: vector (nullable = true)



https://stackoverflow.com/questions/32554527/typeerror-list-indices-must-be-integers-or-slices-not-str
https://stackoverflow.com/questions/50386592/spark-dataframe-aggregate-multiple-column-into-one-column-as-a-string
https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=dataframe
https://spark.apache.org/docs/1.5.0/ml-features.html
https://stackoverflow.com/questions/49832877/adding-a-vectors-column-to-a-pyspark-dataframe
https://stackoverflow.com/questions/42138482/how-do-i-convert-an-array-i-e-list-column-to-vector
http://www.learnbymarketing.com/1100/pyspark-joins-by-example/
https://stackoverflow.com/questions/46944493/removing-duplicate-columns-after-a-df-join-in-spark
https://stackoverflow.com/questions/35067467/how-can-i-get-from-pyspark-sql-types-row-all-the-columns-attributes-name

In [26]:
from pyspark.ml.clustering import LDA

#model = LDA.train(result, k=5, seed=1)
lda = LDA(k=10, maxIter=10, featuresCol = "vectors")
model = lda.fit(result)

KeyboardInterrupt: 

In [ ]:
topics = model.describeTopics(maxTermsPerTopic = 15)
for x, topic in enumerate(topics):
    print('topic nr: ' + str(x))
    words = topic[0]
    weights = topic[1]
    print(len(words))
    print(type(weights))
    for n in range(len(words)):
        print(cvmodel.vocabulary[words[n]] + ' ' + str(weights[n]))